### Mounting Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd 'drive/MyDrive/IIT J Summer Internship 2022/Code/Neural-Uncertainty-Representation/'

/content/drive/.shortcut-targets-by-id/1h_oF16bG32l75GPobgHe_9HgdUduLFxT/IIT J Summer Internship 2022/Code/Neural-Uncertainty-Representation


### Importing Necessary Libraries


In [3]:
import numpy as np
import pandas as pd
from openpyxl import load_workbook

import tensorflow as tf

from tensorflow.keras.layers import Dense, Dropout, LSTM, TimeDistributed
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
import os

### Importing Data
We import the data of a young subject's 3D-Tensor Data of dimensions (TR, Voxel, Number of Subjects) for the 5 different ROIs as follows:
- **Prefrontal areas:** dlPFC, vlPFC, lOFC, dmPFC, mPFC, mOFC
- **Default-mode areas:** mPFC, ACC, PCC, Precuneus, mOFC
- **Sensory areas:** VC
- **Multisensory area:** pSTS
- **Some other areas:** TP, IPL, mCC, Put, PCG, Nac, INS





In [4]:
file_names = [name for name in os.listdir('/content/drive/MyDrive/IIT J Summer Internship 2022/Hitchcock/Data/YOUNG/Voxel_BOLD/Numpy') if name.endswith("npy") and ("(1)" not in name)]
file_names = [name for name in file_names if ("537" not in name)]

In [5]:
ROI_names = [file_name.replace("data_","").replace(".npy","") for file_name in file_names]

In [6]:
data_ori = list()
for file_name in file_names:
  data_ori.append(np.load('/content/drive/MyDrive/IIT J Summer Internship 2022/Hitchcock/Data/YOUNG/Voxel_BOLD/Numpy/'+file_name))

### Data Preparation

Now, here we prepare the data to be fed to the Neural Network Architechtures. We can't predict the time ahead upto which the subjects are predicting correctly so, we will take a bit round about way to predict the BOLD values ahead in time(for several choices of time points e.g. 1 time point ahead, 2 time point ahead, etc) and will compare the results for young and old subjects.

Right from the starting we decide some of the data preparation parameters, so that we can use them as we require.

In [7]:
TRAIN_BATCH_SIZE = 20
TEST_BATCH_SIZE = 10
LOOK_AHEAD = 1
NUM_TEST_SUBS = 10

We will start a basic data preparation where we will average across all the voxels for a ROI and fit the model which predicts this average BOLD value. Later we will avoid doing an average since, all voxels in a ROI is not equally important and hence we will want to predict the BOLD values of several voxels simultaneously.

In [8]:
data = [np.mean(dat, axis=1, keepdims=True) for dat in data_ori]

Let's Start the Data Preparation for lOFC later we will iterate the same procedure for the other ROIs.

In [9]:
print(ROI_names[-1], "Average BOLD Shape:", data[-1].shape)

lOFC Average BOLD Shape: (189, 1, 111)


Below, we see the average BOLD time series plot for all the ROIs.

We have changed the data in the form (Number of Subjects, TR, Voxels) below.

In [10]:
data = [np.transpose(dat, (2,0,1)) for dat in data]

In [11]:
print(ROI_names[-1], "Average BOLD Shape:", data[-1].shape)

lOFC Average BOLD Shape: (111, 189, 1)


We split the dataset of 111 young subjects into 101 subjects in the training data and 10 subject in the test data.

In [12]:
train_data_unscaled = [dat[:-NUM_TEST_SUBS] for dat in data]
test_data_unscaled = [dat[-NUM_TEST_SUBS:] for dat in data]  

We scaled the BOLD values of each voxel in the train data between 0 and 1 for each voxel. We use the same MinMaxScaler fit on the train data on the test data.

In [13]:
scalers = list()
for i in range(len(ROI_names)):
  scalers.append(MinMaxScaler(feature_range = (0, 1)))
train_data = [sc.fit_transform(dat[:,:,0]) for sc, dat in zip(scalers, train_data_unscaled)] 
test_data = [sc.transform(dat[:,:,0]) for sc, dat in zip(scalers, test_data_unscaled)]

In [14]:
train_data = [np.expand_dims(dat, axis = -1) for dat in train_data]
test_data = [np.expand_dims(dat, axis = -1) for dat in test_data]

In [15]:
print(ROI_names[-1], "Train Data Shape: ", train_data[-1].shape)
print(ROI_names[-1], "Test Data Shape: ", test_data[-1].shape)

lOFC Train Data Shape:  (101, 189, 1)
lOFC Test Data Shape:  (10, 189, 1)


We now build a data generator which would produce mini-batches during the training of the neural network.

In [16]:
class KerasBatchGenerator(object):
    def __init__(self, data, num_TRs, batch_size, look_ahead=1):
        self.data = data
        self.num_TRs = num_TRs
        self.batch_size = batch_size
        self.current_idx = 0
        self.look_ahead = look_ahead

    def generate(self):
        while True:
            if self.current_idx + self.batch_size >= self.data.shape[0]:
              self.current_idx = 0
            x = self.data[self.current_idx:(self.current_idx+self.batch_size),:(-self.look_ahead),:]
            y = self.data[self.current_idx:(self.current_idx+self.batch_size),self.look_ahead:,:]
            self.current_idx = self.current_idx + self.batch_size
            yield x, y 

In [17]:
train_data_generator = [KerasBatchGenerator(dat, dat.shape[1], TRAIN_BATCH_SIZE, LOOK_AHEAD) for dat in train_data]
test_data_generator = [KerasBatchGenerator(dat, dat.shape[1], TEST_BATCH_SIZE, LOOK_AHEAD) for dat in test_data]

In [18]:
batch_x, batch_y = next(train_data_generator[-1].generate())
print(ROI_names[-1], "Train Batch X Shape: ", batch_x.shape)
print(ROI_names[-1], "Train Batch y Shape: ", batch_y.shape)

lOFC Train Batch X Shape:  (20, 188, 1)
lOFC Train Batch y Shape:  (20, 188, 1)


In [19]:
batch_x, batch_y = next(test_data_generator[-1].generate())
print(ROI_names[-1], "Test Batch X Shape: ", batch_x.shape)
print(ROI_names[-1], "Test Batch y Shape: ", batch_y.shape)

lOFC Test Batch X Shape:  (10, 188, 1)
lOFC Test Batch y Shape:  (10, 188, 1)


#### Model-1

We will start off by building a LSTM Model:
- `Number of LSTM Layers = 1`
- `Number of Dense Layers = 1`
- `hidden units = 8`
- `dropout = 0.3`

In [20]:
HIDDEN_UNITS = 8
DROPOUT_PROB = 0.3
LSTM_LAYERS = 1
DENSE_LAYERS = 1

In [21]:
def avg_bold_model():
  model = Sequential()
  for _ in range(LSTM_LAYERS):
    model.add(LSTM(HIDDEN_UNITS, return_sequences=True))
  model.add(Dropout(0.3))
  for _ in range(DENSE_LAYERS-1):
    model.add(TimeDistributed(Dense(HIDDEN_UNITS)))
  model.add(TimeDistributed(Dense(1)))
  return model

In [22]:
sample_model = avg_bold_model()
sample_model.build(input_shape=batch_x.shape)
sample_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (10, 188, 8)              320       
                                                                 
 dropout (Dropout)           (10, 188, 8)              0         
                                                                 
 time_distributed (TimeDistr  (10, 188, 1)             9         
 ibuted)                                                         
                                                                 
Total params: 329
Trainable params: 329
Non-trainable params: 0
_________________________________________________________________


In [23]:
models = [avg_bold_model() for _ in range(len(ROI_names))]

In [24]:
for i in range(len(ROI_names)):
  models[i].compile(optimizer = 'adam', loss = 'mean_squared_error')

In [25]:
for i in range(len(ROI_names)):
  models[i].load_weights(f"./Models/YOUNG/LSTM/AVG-BOLD/{ROI_names[i]}/model_weights-{LSTM_LAYERS}L-{HIDDEN_UNITS}H-{DENSE_LAYERS}D-{int(DROPOUT_PROB*10)}Drop-{LOOK_AHEAD}Ahead-{ROI_names[i]}")

In [ ]:
preds = [models[i].predict(next(test_data_generator[i].generate())[0]) for i in range(len(ROI_names))]

In [27]:
a_TRs = np.array([56, 67, 73, 81, 134, 142, 155, 167, 174, 180])
d_TRs = np.array([67, 72, 85, 89, 145, 154, 168, 174, 180, 188])
and_TRs = np.union1d(a_TRs, d_TRs)

In [28]:
def generate_each_sub_n_overall_pred_corr(preds):
  
  print("***Each Test Subject's Correlation Coefficient between predicted and actual average BOLD values of each Ascent and Descent for ROIs***")
  print()
  avg_r = list()
  for i in range(len(ROI_names)):
    df = pd.DataFrame({"Subs":[], "A1":[], "A2":[], "A3":[], "A4":[], "A5":[], "D1":[], "D2":[], "D3":[], "D4":[], "D5":[], "Avg_Asc":[], "Avg_Des":[], "Overall":[]})
    actual_test_data_output = np.squeeze(test_data_unscaled[i])[:,LOOK_AHEAD:] 
    pred_test_data_output = scalers[i].inverse_transform(np.hstack([preds[i][:,:,0], preds[i][:,:LOOK_AHEAD,0]]))[:,:-LOOK_AHEAD]
    for j in range(NUM_TEST_SUBS):
      df_dict = dict()
      df_dict['Subs'] = j+1
      df_dict['Overall'] = np.round(np.corrcoef(actual_test_data_output[j,:], pred_test_data_output[j,:])[0,1], 2)
      sum_as = 0 # stores the sum of each the subject's avg BOLD 5 ascents correlation between actual and preds 
      sum_des = 0 # stores the sum of each the subject's avg BOLD 5 descents correlation between actual and preds 
      for k in range(5):
        a = np.round(np.corrcoef(actual_test_data_output[j,a_TRs[2*k]:a_TRs[2*k+1]], pred_test_data_output[j,a_TRs[2*k]:a_TRs[2*k+1]])[0,1], 2)
        d = np.round(np.corrcoef(actual_test_data_output[j,d_TRs[2*k]:d_TRs[2*k+1]], pred_test_data_output[j,d_TRs[2*k]:d_TRs[2*k+1]])[0,1], 2)
        df_dict['A'+str(k+1)] = a
        df_dict['D'+str(k+1)] = d
        sum_as += a
        sum_des += d
      df_dict['Avg_Asc'] = np.round(sum_as/5, 2)
      df_dict['Avg_Des'] = np.round(sum_des/5, 2)
      df = df.append(df_dict, ignore_index = True)
    print()
    print(f"***{ROI_names[i]}***")
    display(df)
    path = f"./Model-Inference/YOUNG/LSTM/AVG-BOLD/{LSTM_LAYERS}L-{HIDDEN_UNITS}H-{DENSE_LAYERS}D-{int(DROPOUT_PROB*10)}Drop-{LOOK_AHEAD}Ahead-Each-Test-Sub-Corr.xlsx"
    if os.path.isfile(path):  
      with pd.ExcelWriter(path, mode='a') as writer:
        df.to_excel(writer, sheet_name= ROI_names[i])
    else:
      with pd.ExcelWriter(path) as writer:
        df.to_excel(writer, sheet_name= ROI_names[i])



  print("***Average across test subjects Correlation Coefficient between predicted and actual average BOLD values of each Ascent and Descent for ROIs***")
  print()
  df = pd.DataFrame({"ROIs":[], "A1":[], "A2":[], "A3":[], "A4":[], "A5":[], "D1":[], "D2":[], "D3":[], "D4":[], "D5":[], "Avg_Asc":[], "Avg_Des":[], "Overall":[]})
  for i in range(len(ROI_names)):
    actual_test_data_output = np.squeeze(test_data_unscaled[i])[:,LOOK_AHEAD:] 
    pred_test_data_output = scalers[i].inverse_transform(np.hstack([preds[i][:,:,0], preds[i][:,:LOOK_AHEAD,0]]))[:,:-LOOK_AHEAD]
    sum_r = 0 # stores the sum of all the subjects avg BOLD correlation between actual and preds
    sum_a = np.zeros(5) # stores the sum of all the subjects avg BOLD 5 ascents correlation between actual and preds
    sum_d = np.zeros(5) # stores the sum of all the subjects avg BOLD 5 descents between actual and preds
    for j in range(NUM_TEST_SUBS):
      sum_r += np.corrcoef(actual_test_data_output[j,:], pred_test_data_output[j,:])[0,1]
      for k in range(5):
        sum_a[k] += np.corrcoef(actual_test_data_output[j,a_TRs[2*k]:a_TRs[2*k+1]], pred_test_data_output[j,a_TRs[2*k]:a_TRs[2*k+1]])[0,1]
        sum_d[k] += np.corrcoef(actual_test_data_output[j,d_TRs[2*k]:d_TRs[2*k+1]], pred_test_data_output[j,d_TRs[2*k]:d_TRs[2*k+1]])[0,1]
    df.loc[len(df.index)] = [ROI_names[i], *np.round(sum_a / NUM_TEST_SUBS, 2), *np.round(sum_d / NUM_TEST_SUBS, 2), np.round(np.mean(sum_a / NUM_TEST_SUBS), 2), np.round(np.mean(sum_d / NUM_TEST_SUBS), 2), np.round(sum_r / NUM_TEST_SUBS, 2)]
  display(df)
  path = f"./Model-Inference/YOUNG/LSTM/AVG-BOLD/{LSTM_LAYERS}L-{HIDDEN_UNITS}H-{DENSE_LAYERS}D-{int(DROPOUT_PROB*10)}Drop-{LOOK_AHEAD}Ahead-All-ROI-Corr-Avg-Across-Test-Subs.xlsx"
  if os.path.isfile(path):  
    with pd.ExcelWriter(path, mode='a') as writer:
      df.to_excel(writer, sheet_name= ROI_names[i])
  else:
    with pd.ExcelWriter(path) as writer:
      df.to_excel(writer, sheet_name= ROI_names[i])

In [ ]:
generate_each_sub_n_overall_pred_corr(preds)

#### Model-2

We will start off by building a LSTM Model:
- `Number of LSTM Layers = 1`
- `Number of Dense Layers = 1`
- `hidden units = 16`
- `dropout = 0.3`

In [30]:
HIDDEN_UNITS = 16
DROPOUT_PROB = 0.3
LSTM_LAYERS = 1
DENSE_LAYERS = 1

In [31]:
sample_model = avg_bold_model()
sample_model.build(input_shape=batch_x.shape)
sample_model.summary()

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_21 (LSTM)              (10, 188, 16)             1152      
                                                                 
 dropout_21 (Dropout)        (10, 188, 16)             0         
                                                                 
 time_distributed_21 (TimeDi  (10, 188, 1)             17        
 stributed)                                                      
                                                                 
Total params: 1,169
Trainable params: 1,169
Non-trainable params: 0
_________________________________________________________________


In [32]:
models = [avg_bold_model() for _ in range(len(ROI_names))]

In [ ]:
for i in range(len(ROI_names)):
  models[i].compile(optimizer = 'adam', loss = 'mean_squared_error')

In [34]:
for i in range(len(ROI_names)):
  models[i].load_weights(f"./Models/YOUNG/LSTM/AVG-BOLD/{ROI_names[i]}/model_weights-{LSTM_LAYERS}L-{HIDDEN_UNITS}H-{DENSE_LAYERS}D-0{int(DROPOUT_PROB*10)}Drop-{LOOK_AHEAD}Ahead-{ROI_names[i]}")

In [35]:
preds = [models[i].predict(next(test_data_generator[i].generate())[0]) for i in range(len(ROI_names))]

In [ ]:
generate_each_sub_n_overall_pred_corr(preds)

#### Model-3

We will start off by building a LSTM Model:
- `Number of LSTM Layers = 1`
- `Number of Dense Layers = 1`
- `hidden units = 32`
- `dropout = 0.3`

In [37]:
HIDDEN_UNITS = 32
DROPOUT_PROB = 0.3
LSTM_LAYERS = 1
DENSE_LAYERS = 1

In [38]:
sample_model = avg_bold_model()
sample_model.build(input_shape=batch_x.shape)
sample_model.summary()

Model: "sequential_42"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_42 (LSTM)              (10, 188, 32)             4352      
                                                                 
 dropout_42 (Dropout)        (10, 188, 32)             0         
                                                                 
 time_distributed_42 (TimeDi  (10, 188, 1)             33        
 stributed)                                                      
                                                                 
Total params: 4,385
Trainable params: 4,385
Non-trainable params: 0
_________________________________________________________________


In [39]:
models = [avg_bold_model() for _ in range(len(ROI_names))]

In [40]:
for i in range(len(ROI_names)):
  models[i].compile(optimizer = 'adam', loss = 'mean_squared_error')

In [41]:
for i in range(len(ROI_names)):
  models[i].load_weights(f"./Models/YOUNG/LSTM/AVG-BOLD/{ROI_names[i]}/model_weights-{LSTM_LAYERS}L-{HIDDEN_UNITS}H-{DENSE_LAYERS}D-{int(DROPOUT_PROB*10)}Drop-{LOOK_AHEAD}Ahead-{ROI_names[i]}")

In [ ]:
preds = [models[i].predict(next(test_data_generator[i].generate())[0]) for i in range(len(ROI_names))]

In [ ]:
generate_each_sub_n_overall_pred_corr(preds)

#### Model-4

We will start off by building a LSTM Model:
- `Number of LSTM Layers = 2`
- `Number of Dense Layers = 1`
- `hidden units = 16`
- `dropout = 0.3`

In [44]:
HIDDEN_UNITS = 16
DROPOUT_PROB = 0.3
LSTM_LAYERS = 2
DENSE_LAYERS = 1

In [45]:
sample_model = avg_bold_model()
sample_model.build(input_shape=batch_x.shape)
sample_model.summary()

Model: "sequential_63"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_63 (LSTM)              (10, 188, 16)             1152      
                                                                 
 lstm_64 (LSTM)              (10, 188, 16)             2112      
                                                                 
 dropout_63 (Dropout)        (10, 188, 16)             0         
                                                                 
 time_distributed_63 (TimeDi  (10, 188, 1)             17        
 stributed)                                                      
                                                                 
Total params: 3,281
Trainable params: 3,281
Non-trainable params: 0
_________________________________________________________________


In [46]:
models = [avg_bold_model() for _ in range(len(ROI_names))]

In [47]:
for i in range(len(ROI_names)):
  models[i].compile(optimizer = 'adam', loss = 'mean_squared_error')

In [48]:
for i in range(len(ROI_names)):
  models[i].load_weights(f"./Models/YOUNG/LSTM/AVG-BOLD/{ROI_names[i]}/model_weights-{LSTM_LAYERS}L-{HIDDEN_UNITS}H-{DENSE_LAYERS}D-{int(DROPOUT_PROB*10)}Drop-{LOOK_AHEAD}Ahead-{ROI_names[i]}")

In [ ]:
preds = [models[i].predict(next(test_data_generator[i].generate())[0]) for i in range(len(ROI_names))]

In [ ]:
generate_each_sub_n_overall_pred_corr(preds)

#### Model-5

We will start off by building a LSTM Model:
- `Number of LSTM Layers = 2`
- `Number of Dense Layers = 2`
- `hidden units = 32`
- `dropout = 0.3`

In [51]:
HIDDEN_UNITS = 32
DROPOUT_PROB = 0.3
LSTM_LAYERS = 2
DENSE_LAYERS = 2

In [52]:
sample_model = avg_bold_model()
sample_model.build(input_shape=batch_x.shape)
sample_model.summary()

Model: "sequential_84"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_105 (LSTM)             (10, 188, 32)             4352      
                                                                 
 lstm_106 (LSTM)             (10, 188, 32)             8320      
                                                                 
 dropout_84 (Dropout)        (10, 188, 32)             0         
                                                                 
 time_distributed_84 (TimeDi  (10, 188, 32)            1056      
 stributed)                                                      
                                                                 
 time_distributed_85 (TimeDi  (10, 188, 1)             33        
 stributed)                                                      
                                                                 
Total params: 13,761
Trainable params: 13,761
Non-tra

In [53]:
models = [avg_bold_model() for _ in range(len(ROI_names))]

In [54]:
for i in range(len(ROI_names)):
  models[i].compile(optimizer = 'adam', loss = 'mean_squared_error')

In [55]:
for i in range(len(ROI_names)):
  models[i].load_weights(f"./Models/YOUNG/LSTM/AVG-BOLD/{ROI_names[i]}/model_weights-{LSTM_LAYERS}L-{HIDDEN_UNITS}H-{DENSE_LAYERS}D-{int(DROPOUT_PROB*10)}Drop-{LOOK_AHEAD}Ahead-{ROI_names[i]}")

In [ ]:
preds = [models[i].predict(next(test_data_generator[i].generate())[0]) for i in range(len(ROI_names))]

In [ ]:
generate_each_sub_n_overall_pred_corr(preds)

#### Model-6

We will start off by building a LSTM Model:
- `Number of LSTM Layers = 1`
- `Number of Dense Layers = 2`
- `hidden units = 32`
- `dropout = 0.3`

In [58]:
HIDDEN_UNITS = 32
DROPOUT_PROB = 0.3
LSTM_LAYERS = 1
DENSE_LAYERS = 2

In [59]:
sample_model = avg_bold_model()
sample_model.build(input_shape=batch_x.shape)
sample_model.summary()

Model: "sequential_105"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_147 (LSTM)             (10, 188, 32)             4352      
                                                                 
 dropout_105 (Dropout)       (10, 188, 32)             0         
                                                                 
 time_distributed_126 (TimeD  (10, 188, 32)            1056      
 istributed)                                                     
                                                                 
 time_distributed_127 (TimeD  (10, 188, 1)             33        
 istributed)                                                     
                                                                 
Total params: 5,441
Trainable params: 5,441
Non-trainable params: 0
_________________________________________________________________


In [60]:
models = [avg_bold_model() for _ in range(len(ROI_names))]

In [61]:
for i in range(len(ROI_names)):
  models[i].compile(optimizer = 'adam', loss = 'mean_squared_error')

In [62]:
for i in range(len(ROI_names)):
  models[i].load_weights(f"./Models/YOUNG/LSTM/AVG-BOLD/{ROI_names[i]}/model_weights-{LSTM_LAYERS}L-{HIDDEN_UNITS}H-{DENSE_LAYERS}D-{int(DROPOUT_PROB*10)}Drop-{LOOK_AHEAD}Ahead-{ROI_names[i]}")

In [ ]:
preds = [models[i].predict(next(test_data_generator[i].generate())[0]) for i in range(len(ROI_names))]

In [ ]:
generate_each_sub_n_overall_pred_corr(preds)

#### Model-7

We will start off by building a LSTM Model:
- `Number of LSTM Layers = 1`
- `Number of Dense Layers = 4`
- `hidden units = 16`
- `dropout = 0.3`

In [65]:
HIDDEN_UNITS = 16
DROPOUT_PROB = 0.3
LSTM_LAYERS = 1
DENSE_LAYERS = 4

In [66]:
sample_model = avg_bold_model()
sample_model.build(input_shape=batch_x.shape)
sample_model.summary()

Model: "sequential_126"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_168 (LSTM)             (10, 188, 16)             1152      
                                                                 
 dropout_126 (Dropout)       (10, 188, 16)             0         
                                                                 
 time_distributed_168 (TimeD  (10, 188, 16)            272       
 istributed)                                                     
                                                                 
 time_distributed_169 (TimeD  (10, 188, 16)            272       
 istributed)                                                     
                                                                 
 time_distributed_170 (TimeD  (10, 188, 16)            272       
 istributed)                                                     
                                                    

In [67]:
models = [avg_bold_model() for _ in range(len(ROI_names))]

In [68]:
for i in range(len(ROI_names)):
  models[i].compile(optimizer = 'adam', loss = 'mean_squared_error')

In [69]:
for i in range(len(ROI_names)):
  models[i].load_weights(f"./Models/YOUNG/LSTM/AVG-BOLD/{ROI_names[i]}/model_weights-{LSTM_LAYERS}L-{HIDDEN_UNITS}H-{DENSE_LAYERS}D-{int(DROPOUT_PROB*10)}Drop-{LOOK_AHEAD}Ahead-{ROI_names[i]}")

In [ ]:
preds = [models[i].predict(next(test_data_generator[i].generate())[0]) for i in range(len(ROI_names))]

In [ ]:
generate_each_sub_n_overall_pred_corr(preds)

#### Model-8

We will start off by building a LSTM Model:
- `Number of LSTM Layers = 1`
- `Number of Dense Layers = 3`
- `hidden units = 16`
- `dropout = 0.3`

In [72]:
HIDDEN_UNITS = 16
DROPOUT_PROB = 0.3
LSTM_LAYERS = 1
DENSE_LAYERS = 3

In [73]:
sample_model = avg_bold_model()
sample_model.build(input_shape=batch_x.shape)
sample_model.summary()

Model: "sequential_147"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_189 (LSTM)             (10, 188, 16)             1152      
                                                                 
 dropout_147 (Dropout)       (10, 188, 16)             0         
                                                                 
 time_distributed_252 (TimeD  (10, 188, 16)            272       
 istributed)                                                     
                                                                 
 time_distributed_253 (TimeD  (10, 188, 16)            272       
 istributed)                                                     
                                                                 
 time_distributed_254 (TimeD  (10, 188, 1)             17        
 istributed)                                                     
                                                    

In [74]:
models = [avg_bold_model() for _ in range(len(ROI_names))]

In [75]:
for i in range(len(ROI_names)):
  models[i].compile(optimizer = 'adam', loss = 'mean_squared_error')

In [76]:
for i in range(len(ROI_names)):
  models[i].load_weights(f"./Models/YOUNG/LSTM/AVG-BOLD/{ROI_names[i]}/model_weights-{LSTM_LAYERS}L-{HIDDEN_UNITS}H-{DENSE_LAYERS}D-{int(DROPOUT_PROB*10)}Drop-{LOOK_AHEAD}Ahead-{ROI_names[i]}")

In [ ]:
preds = [models[i].predict(next(test_data_generator[i].generate())[0]) for i in range(len(ROI_names))]

In [ ]:
generate_each_sub_n_overall_pred_corr(preds)

#### Model-9

We will start off by building a LSTM Model:
- `Number of LSTM Layers = 1`
- `Number of Dense Layers = 2`
- `hidden units = 16`
- `dropout = 0.3`

In [79]:
HIDDEN_UNITS = 16
DROPOUT_PROB = 0.3
LSTM_LAYERS = 1
DENSE_LAYERS = 2

In [80]:
sample_model = avg_bold_model()
sample_model.build(input_shape=batch_x.shape)
sample_model.summary()

Model: "sequential_168"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_210 (LSTM)             (10, 188, 16)             1152      
                                                                 
 dropout_168 (Dropout)       (10, 188, 16)             0         
                                                                 
 time_distributed_315 (TimeD  (10, 188, 16)            272       
 istributed)                                                     
                                                                 
 time_distributed_316 (TimeD  (10, 188, 1)             17        
 istributed)                                                     
                                                                 
Total params: 1,441
Trainable params: 1,441
Non-trainable params: 0
_________________________________________________________________


In [81]:
models = [avg_bold_model() for _ in range(len(ROI_names))]

In [82]:
for i in range(len(ROI_names)):
  models[i].compile(optimizer = 'adam', loss = 'mean_squared_error')

In [83]:
for i in range(len(ROI_names)):
  models[i].load_weights(f"./Models/YOUNG/LSTM/AVG-BOLD/{ROI_names[i]}/model_weights-{LSTM_LAYERS}L-{HIDDEN_UNITS}H-{DENSE_LAYERS}D-{int(DROPOUT_PROB*10)}Drop-{LOOK_AHEAD}Ahead-{ROI_names[i]}")

In [ ]:
preds = [models[i].predict(next(test_data_generator[i].generate())[0]) for i in range(len(ROI_names))]

In [ ]:
generate_each_sub_n_overall_pred_corr(preds)